In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
import evaluate

# Parametri
max_input_length = 512
max_target_length = 512


In [3]:
dataset_id = "1381trsIgBnScOVM7vVAAWw2jP-jiZFQ0" #dataset da 160 righe
url = f"https://drive.google.com/uc?id={dataset_id}"


In [4]:
import pandas as pd
from datasets import Dataset

#df = pd.read_csv("traduzione_volgare.csv")
df = pd.read_csv(url)

#Converting pandas dataset into huggingface dataset
dataset = Dataset.from_pandas(df)


In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [7]:
source_lang = "it_IT"        # Italian
target_lang = "it_IT"        # Italian
tokenizer.src_lang = source_lang


In [8]:
print(dataset[0])
print(dataset['Sentence'])
print(dataset['translation'])

{'Sentence': 'Nel mezzo del cammin di nostra vita mi ritrovai per una selva oscura, ché la diritta via era smarrita.', 'translation': 'A metà del percorso della vita umana, mi ritrovai per una oscura foresta, poiché avevo smarrito la giusta strada.'}
['Nel mezzo del cammin di nostra vita mi ritrovai per una selva oscura, ché la diritta via era smarrita.', 'Ahi quanto a dir qual era è cosa dura esta selva selvaggia e aspra e forte che nel pensier rinova la paura!', 'Tant’è amara che poco è più morte; ma per trattar del ben ch’i’ vi trovai, dirò de l’altre cose ch’i’ v’ho scorte.', 'Io non so ben ridir com’i’ v’intrai, tant’era pien di sonno a quel punto che la verace via abbandonai.', 'Ma poi ch’i’ fui al piè d’un colle giunto, là dove terminava quella valle che m’avea di paura il cor compunto, guardai in alto e vidi le sue spalle vestite già de’ raggi del pianeta che mena dritto altrui per ogne calle. ', 'Allor fu la paura un poco queta, che nel lago del cor m’era durata la notte ch’i’

In [9]:
#Tokenizing the Dataset

def safe_str(x):
    return str(x) if x is not None else ""

def preprocess_function(examples):
    # Safely get inputs and targets
    inputs = examples.get("Sentence", [])
    targets = examples.get("translation", [])

    # Ensure they're lists
    if isinstance(inputs, str):
        inputs = [inputs]
    if isinstance(targets, str):
        targets = [targets]

    # Apply safe_str to each
    inputs = [safe_str(s) for s in inputs]
    targets = [safe_str(t) for t in targets]

    # Tokenize source text
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        targets,
        max_length=max_target_length,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Applica la tokenizzazione
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Visualizza i primi esempi tokenizzati
print(tokenized_dataset[:3])




Map:   0%|          | 0/160 [00:00<?, ? examples/s]

{'Sentence': ['Nel mezzo del cammin di nostra vita mi ritrovai per una selva oscura, ché la diritta via era smarrita.', 'Ahi quanto a dir qual era è cosa dura esta selva selvaggia e aspra e forte che nel pensier rinova la paura!', 'Tant’è amara che poco è più morte; ma per trattar del ben ch’i’ vi trovai, dirò de l’altre cose ch’i’ v’ho scorte.'], 'translation': ['A metà del percorso della vita umana, mi ritrovai per una oscura foresta, poiché avevo smarrito la giusta strada.', "Ahimè, è difficile descrivere com'era quella foresta, selvaggia, inestricabile e tremenda, tale che al solo pensiero fa tornare la paura.", 'È così spaventosa che la morte lo è poco di più: ma per descrivere il bene che vi trovai dentro, dirò quali altre cose ho visto in essa.'], 'input_ids': [[250011, 12940, 57793, 146, 7384, 1249, 45, 6337, 4971, 324, 120030, 14, 117, 220, 2432, 11, 362, 27315, 4, 6, 12640, 21, 5121, 2310, 1829, 1615, 94778, 13992, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [10]:
from sklearn.model_selection import train_test_split

split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

In [11]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=734801ba614105e984c2303c6af6e241954fcbd8990f58b76737ba91df4dc92b
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [12]:
import numpy as np


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Replace -100 in labels as they are ignored during loss computation
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v, 4) for k, v in result.items()}


In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
)


In [14]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



model.config.forced_bos_token_id = tokenizer.convert_tokens_to_ids(target_lang)
trainer.train()

<ipython-input-14-18adeec28028>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: giuminia2000 (giuminia2000-sapienza-universit-di-roma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,11.380500,9.315193,0.357100,0.123600,0.312800,0.310400
2,8.551700,7.812809,0.393600,0.151400,0.343600,0.341300
3,7.719400,6.623630,0.383500,0.140800,0.341600,0.339200
4,6.382100,5.833095,0.394000,0.154500,0.348300,0.346000
5,5.695400,5.486613,0.396600,0.148600,0.347200,0.345800


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1730: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3464: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_bos_token_id': 250011}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=80, training_loss=7.774563980102539, metrics={'train_runtime': 104.8978, 'train_samples_per_second': 6.101, 'train_steps_per_second': 0.763, 'total_flos': 693481525739520.0, 'train_loss': 7.774563980102539, 'epoch': 5.0})

In [15]:
#Prova
frase = "Officio di questa arte pare che sia dicere appostatamente per fare credere, fine è far credere per lo dire."

input_text = f"translate: {frase}"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

output_ids = model.generate(input_ids, max_length=128, num_beams=4)
traduzione = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Moderno:", traduzione)


Moderno: L'intento di questa arte sembra essere solo per far credere, il fine è far credere per dire.


In [16]:
# save the model
trainer.save_model("./fine_tuning_volgare")
tokenizer.save_pretrained("./fine_tuning_volgare")

('./fine_tuning_volgare/tokenizer_config.json',
 './fine_tuning_volgare/special_tokens_map.json',
 './fine_tuning_volgare/sentencepiece.bpe.model',
 './fine_tuning_volgare/added_tokens.json',
 './fine_tuning_volgare/tokenizer.json')

In [17]:
model.save_pretrained("./fine_tuning_volgare")

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
trainer.save_model("/content/drive/MyDrive/fine_tuning_volgare")
model.save_pretrained("/content/drive/MyDrive/fine_tuning_volgare")
tokenizer.save_pretrained("/content/drive/MyDrive/fine_tuning_volgare")

('/content/drive/MyDrive/fine_tuning_volgare/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuning_volgare/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuning_volgare/sentencepiece.bpe.model',
 '/content/drive/MyDrive/fine_tuning_volgare/added_tokens.json',
 '/content/drive/MyDrive/fine_tuning_volgare/tokenizer.json')